In [27]:
import requests
import json
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
from shapely.geometry import shape
import rasterio as rio
from rasterio.features import dataset_features
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely import wkt
# import rioxarray as rxr
# import osmnx as ox

In [94]:
DATA_PATH = "../data"

In [66]:
def load_osm_result(path):
    with open(path, "r") as f:
        osm_response = json.load(f)

    data = []
    for element in osm_response["elements"]:
        if element["type"] == "node" and "lat" in element and "lon" in element:
            properties = element.get("tags", {})
            properties.update({"id": element["id"], "lat": element["lat"], "lon": element["lon"]})
            data.append(properties)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
    gdf.set_crs(epsg=4326, inplace=True)# Set the CRS to WGS84 (EPSG:4326)
    return gdf

In [96]:
population_0_path = os.path.join(DATA_PATH, "population/Bevölkerung 0 bis 5 Jahre.csv")
population_1_path = os.path.join(DATA_PATH, "population/Bevölkerung 60 bis 74 Jahre.csv")
population_2_path = os.path.join(DATA_PATH, "population/Bevölkerung ab 75 Jahre.csv")

try:
    df_0 = pd.read_csv(
        population_0_path,
        sep=';',
        # on_bad_lines='skip' # Überspringt fehlerhafte Zeilen
      ) 
    df_0[['srid', 'geom']] = [element.split(';') for element in df_0['geom']]
    df_0['geometry'] = df_0['geom'].apply(wkt.loads)
    df_0.rename(columns={"prozent": "prozent_0_5"}, inplace=True)
    print(df_0.shape)
except Exception as e:
    print(f"Fehler beim Laden der CSV-Datei: {e}")
try:
    df_1 = pd.read_csv(
        population_1_path,
        sep=';',
        # on_bad_lines='skip' # Überspringt fehlerhafte Zeilen
      )  
    df_1[['srid', 'geom']] = [element.split(';') for element in df_1['geom']]
    df_1['geometry'] = df_1['geom'].apply(wkt.loads)
    df_1.rename(columns={"prozent": "prozent_60_74"}, inplace=True)
    print(df_1.shape)
except Exception as e:
    print(f"Fehler beim Laden der CSV-Datei: {e}")
try:
    df_2 = pd.read_csv(
        population_2_path,
        sep=';',
        # on_bad_lines='skip' # Überspringt fehlerhafte Zeilen
      ) 
    df_2[['srid', 'geom']] = [element.split(';') for element in df_2['geom']]
    df_2['geometry'] = df_2['geom'].apply(wkt.loads)
    df_2.rename(columns={"prozent": "prozent_75_"}, inplace=True)
    print(df_2.shape)
except Exception as e:
    print(f"Fehler beim Laden der CSV-Datei: {e}")

(61, 9)
(61, 9)
(61, 9)


In [97]:
df_population = df_0[['id', 'geometry', 'prozent_0_5']].merge(df_1[['id', 'prozent_60_74']], on='id').merge(df_2[['id', 'prozent_75_']], on='id')
# .merge(df_2[['id', 'prozent_74_']].set_index['id'])
df_population.drop(columns=['id'], inplace=True)
gdf_population = gpd.GeoDataFrame(df_population, geometry='geometry')
gdf_population.crs = 'EPSG:4326'
gdf_population

,geometry,prozent_0_5,prozent_60_74,prozent_75_
0,"POLYGON ((13.73990 51.04697, 13.73968 51.04699...",4.67,10.58,11.25
1,"POLYGON ((13.75017 51.04619, 13.74987 51.04568...",5.12,11.62,17.84
2,"POLYGON ((13.77594 51.03774, 13.77595 51.03774...",4.08,11.27,16.51
3,"POLYGON ((13.73231 51.05056, 13.73236 51.05028...",5.30,12.41,18.16
4,"POLYGON ((13.72903 51.06570, 13.72951 51.06522...",7.01,7.29,6.12
...,...,...,...,...
56,"POLYGON ((13.66609 51.04002, 13.66572 51.04003...",5.36,19.26,9.75
57,"POLYGON ((13.68108 51.04884, 13.68123 51.04882...",4.81,22.52,14.12
58,"POLYGON ((13.67230 51.05024, 13.67229 51.05001...",7.07,21.73,6.72
59,"POLYGON ((13.67378 51.05068, 13.67368 51.05066...",5.14,19.22,11.34


In [59]:
def raster_layer_to_vector(rasterfile):
     
    with rio.open(rasterfile, mode="r") as dataset:
        arr = dataset.read()[0,:,:]

    #If your input raster have many values, each unique value will become one polygon.
    #Reclassify to either 1 or 0. You dont have to do this.
    arr[(arr>0)]=1
    arr[(arr<1)]=0

    #Vectorize each raster blob
    mask = arr>0 #Dont create polygons where the value is <=0
    shapes = rio.features.shapes(source=arr, connectivity=4, mask=mask,
                                                transform=dataset.transform)

    #Iterate over the shapes, each row is a tuple of the geometry and the raster value:
    #({'type': 'Polygon', 'coordinates': [[(618821.3726, 6480531.5759), ....), 1.0)

    rastervalues = [] #A list for the values
    geometries = [] #And geometries
    for geom, rasterval in shapes:
        geometries.append(shape(geom)) #Create a shapely polygon
        rastervalues.append(rasterval)

    df = gpd.GeoDataFrame(data=rastervalues, geometry=geometries, 
                        crs=dataset.crs.to_epsg(), columns=["rasterval"])

    df.drop("rasterval", axis=1, inplace=True)
    return df

In [63]:
stufe_1_path = "../../data/interim/landsat/LC09_L1TP_192024_20220719_20230406_02_T1/stufe_1.TIF"
stufe_1 = raster_layer_to_vector(stufe_1_path).to_crs(epsg=4326)
stufe_1["stufe"] = 1

stufe_2_path = "../../data/interim/landsat/LC09_L1TP_192024_20220719_20230406_02_T1/stufe_2.TIF"
stufe_2 = raster_layer_to_vector(stufe_2_path).to_crs(epsg=4326)
stufe_2["stufe"] = 2

stufe_3_path = "../../data/interim/landsat/LC09_L1TP_192024_20220719_20230406_02_T1/stufe_3.TIF"
stufe_3 = raster_layer_to_vector(stufe_3_path).to_crs(epsg=4326)
stufe_3["stufe"] = 3

stufen = pd.concat([stufe_1, stufe_2, stufe_3])
interim_path = os.path.join(DATA_PATH, "interim/stufen.json")
stufen.to_file(interim_path, driver="GeoJSON")  

In [98]:
bus_stops_path = os.path.join(DATA_PATH, "raw/bus_stops_dresden.json")
bus_stops= load_osm_result(bus_stops_path).loc[:, ["bench", "shelter", "name", "network", "public_transport", "geometry"]]
bus_stops

,bench,shelter,name,network,public_transport,geometry
0,yes,no,Jeschütz Dorfplatz,ZVON,platform,POINT (14.46204 51.23395)
1,NaN,NaN,"Freital, Burgker Straße (2)",NaN,stop_position,POINT (13.68611 51.00311)
2,NaN,NaN,Görlitz Friesenstraße,ZVON,stop_position,POINT (14.95194 51.13773)
3,NaN,NaN,Görlitz J-S-Bach-Straße,ZVON,stop_position,POINT (14.96194 51.14021)
4,NaN,NaN,Radebeul Hainstraße,NaN,platform,POINT (13.63664 51.10475)
...,...,...,...,...,...,...
9919,no,no,Oppach Busbahnhof,ZVON,platform,POINT (14.49938 51.05645)
9920,yes,no,Olbersdorf Schule,ZVON,platform,POINT (14.77400 50.87375)
9921,yes,no,Olbersdorf Schule,ZVON,platform,POINT (14.77416 50.87389)
9922,yes,no,Olbersdorf Schule,ZVON,platform,POINT (14.77428 50.87386)


# Split bus stops by risk

In [99]:
bus_stops = gpd.sjoin(bus_stops, stufen, how="inner", predicate="within")
bus_stops.rename(columns={"index_right": "index_stufen"}, inplace=True)
bus_stops

,bench,shelter,name,network,public_transport,geometry,index_stufen,stufe
48,no,no,Keppgrundstraße,Verkehrsverbund Oberelbe,platform,POINT (13.84861 51.00984),408,2
49,no,no,Keppgrundstraße,Verkehrsverbund Oberelbe,platform,POINT (13.84846 51.01024),408,2
50,no,no,Schiffswerft Laubegast,NaN,platform,POINT (13.84267 51.01843),245,3
51,yes,yes,Schiffswerft Laubegast,NaN,platform,POINT (13.84184 51.01958),408,2
52,yes,no,"Laubegast, Kronstädter Platz",Verkehrsverbund Oberelbe,platform,POINT (13.83954 51.02245),408,2
...,...,...,...,...,...,...,...,...
9824,no,no,Burgenlandstraße,NaN,platform,POINT (13.82777 51.02502),408,2
9836,NaN,NaN,NaN,NaN,platform,POINT (13.83587 51.02448),408,2
9837,NaN,no,Tolkewitz Schulcampus,NaN,platform,POINT (13.81759 51.03489),408,2
9838,NaN,no,Tolkewitz Schulcampus,NaN,platform,POINT (13.81693 51.03423),408,2


In [100]:
bus_stops = gpd.sjoin(bus_stops, gdf_population, how="inner", predicate="within")
bus_stops.rename(columns={"index_right": "index_population"}, inplace=True)
bus_stops

,bench,shelter,name,network,public_transport,geometry,index_stufen,stufe,index_population,prozent_0_5,prozent_60_74,prozent_75_
48,no,no,Keppgrundstraße,Verkehrsverbund Oberelbe,platform,POINT (13.84861 51.00984),408,2,36,4.32,20.12,15.02
49,no,no,Keppgrundstraße,Verkehrsverbund Oberelbe,platform,POINT (13.84846 51.01024),408,2,36,4.32,20.12,15.02
50,no,no,Schiffswerft Laubegast,NaN,platform,POINT (13.84267 51.01843),245,3,35,4.46,20.22,15.48
51,yes,yes,Schiffswerft Laubegast,NaN,platform,POINT (13.84184 51.01958),408,2,35,4.46,20.22,15.48
52,yes,no,"Laubegast, Kronstädter Platz",Verkehrsverbund Oberelbe,platform,POINT (13.83954 51.02245),408,2,35,4.46,20.22,15.48
...,...,...,...,...,...,...,...,...,...,...,...,...
9824,no,no,Burgenlandstraße,NaN,platform,POINT (13.82777 51.02502),408,2,35,4.46,20.22,15.48
9836,NaN,NaN,NaN,NaN,platform,POINT (13.83587 51.02448),408,2,35,4.46,20.22,15.48
9837,NaN,no,Tolkewitz Schulcampus,NaN,platform,POINT (13.81759 51.03489),408,2,31,5.46,16.53,15.20
9838,NaN,no,Tolkewitz Schulcampus,NaN,platform,POINT (13.81693 51.03423),408,2,31,5.46,16.53,15.20


In [135]:
interim_path = os.path.join(DATA_PATH, "interim/bus_stops_enhanced.json")
bus_stops.to_file(interim_path, driver="GeoJSON")  
bus_stops

,bench,shelter,name,network,public_transport,geometry,index_right,stufe
48,no,no,Keppgrundstraße,Verkehrsverbund Oberelbe,platform,POINT (13.84861 51.00984),408,2
49,no,no,Keppgrundstraße,Verkehrsverbund Oberelbe,platform,POINT (13.84846 51.01024),408,2
51,yes,yes,Schiffswerft Laubegast,NaN,platform,POINT (13.84184 51.01958),408,2
52,yes,no,"Laubegast, Kronstädter Platz",Verkehrsverbund Oberelbe,platform,POINT (13.83954 51.02245),408,2
53,yes,yes,"Laubegast, Kronstädter Platz",Verkehrsverbund Oberelbe,platform,POINT (13.83922 51.02268),408,2
...,...,...,...,...,...,...,...,...
8341,no,no,Gomlitzer Höhe,NaN,platform,POINT (13.78920 51.15214),21,3
8369,no,no,Radeburger Landstraße,NaN,platform,POINT (13.79421 51.15293),21,3
8368,no,no,Promigberg,NaN,platform,POINT (13.79983 51.16079),7,3
9655,no,no,Rochwitzer Straße,NaN,platform,POINT (13.85867 51.05133),207,2


In [108]:
bus_stops.groupby(["shelter", "stufe"]).count()

bench  name  network  public_transport  geometry  index_right
shelter stufe                                                               
no      1         93    94       16                93        95           95
        2        773   782       82               768       784          784
        3         18    18        1                18        18           18
yes     1         51    51        7                51        51           51
        2        461   466       53               454       466          466
        3          5     5        0                 5         5            5

In [115]:
def style_function(feature):
    category = feature['properties']['stufe']
    if category == 1:
        color = '#b7f598' # green
    elif category == 2:
        color = '#f5d998' # orange
    elif category == 3:
        color = '#f59c98'  # red
    
    return {
        'fillColor': color,
        'color': color,
        'weight': 1,
        'fillOpacity': 1
    }
    
combined_geom = stufen.dissolve().to_crs("epsg:4326")
centroid = combined_geom.geometry.centroid
centroid_point = centroid.iloc[0]
longitude = centroid.x.values[0]
latitude = centroid.y.values[0]

m = folium.Map(location=[latitude, longitude], zoom_start=11, tiles="cartodb positron")

# Add the polygons to the map
folium.GeoJson(stufen, style_function=style_function, interactive=True,name="Belastungsstufen").add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

/tmp/ipykernel_41873/3830667383.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = combined_geom.geometry.centroid


In [78]:
"""bus_stops = pd.read_csv("dresden_bus_stops.csv")
"""for c in bus_stops.columns:
    print(c)"""
bus_stops = bus_stops.loc[:, ["bench", "shelter", "name", "network", "public_transport", "lat", "lon", "geometry"]]
bus_stops.loc[:, ["bench", "shelter", "name", "network", "public_transport", "lat", "lon"]]
bus_stops['geometry'] = bus_stops.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
bus_stops = gpd.GeoDataFrame(bus_stops, geometry="geometry")

bus_stops.set_crs(epsg=4326, inplace=True)"""

/tmp/ipykernel_41873/3574125099.py:1: DtypeWarning: Columns (19,26,34,35,37,40,43,45,46,48,49,53,57,58,60,62,64,65,69,72,74,76,77,78,79,80,82,84,85,86,88,89,90,91,93,94,95,97,99,100,101,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  bus_stops = pd.read_csv("dresden_bus_stops.csv")
/usr/local/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:130: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,bench,shelter,name,network,public_transport,lat,lon,geometry
0,yes,no,Jeschütz Dorfplatz,ZVON,platform,51.233948,14.462044,POINT (14.46204 51.23395)
1,NaN,NaN,"Freital, Burgker Straße (2)",NaN,stop_position,51.003110,13.686115,POINT (13.68611 51.00311)
2,NaN,NaN,Görlitz Friesenstraße,ZVON,stop_position,51.137725,14.951937,POINT (14.95194 51.13773)
3,NaN,NaN,Görlitz J-S-Bach-Straße,ZVON,stop_position,51.140205,14.961944,POINT (14.96194 51.14021)
4,NaN,NaN,Radebeul Hainstraße,NaN,platform,51.104751,13.636639,POINT (13.63664 51.10475)
...,...,...,...,...,...,...,...,...
9919,no,no,Oppach Busbahnhof,ZVON,platform,51.056448,14.499384,POINT (14.49938 51.05645)
9920,yes,no,Olbersdorf Schule,ZVON,platform,50.873750,14.773997,POINT (14.77400 50.87375)
9921,yes,no,Olbersdorf Schule,ZVON,platform,50.873894,14.774157,POINT (14.77416 50.87389)
9922,yes,no,Olbersdorf Schule,ZVON,platform,50.873859,14.774277,POINT (14.77428 50.87386)


Try this approach later https://discourse.pangeo.io/t/intersecting-shapefiles-and-raster-data-at-scale-good-design-patterns/1188/2